In [1]:
from fastai.vision.all import *
from fastai.tabular.all import *

In [2]:
PATH = Path('../input/optiver-realized-volatility-prediction')

# data_df = pd.read_parquet(PATH/'book_train_reindexed.parquet')

# sizes = ['bid_size1', 'bid_size2', 'ask_size1', 'ask_size2']
# for sz in sizes:
#     data_df[sz] = data_df[sz]/ 1_000_000



In [3]:
#train_df = pd.read_csv(PATH/'train_with_offset.csv')
# data_df['wap'] = (data_df.bid_price1 * data_df.ask_size1 + data_df.ask_price1 * data_df.bid_size1) / (data_df.ask_size1 + data_df.bid_size1)

# mean_wap =[]
# for row in train_df.itertuples():
#     df = data_df.iloc[row.offset:row.offset+600]
#     mean_wap.append(df.wap.mean())

In [4]:
# train_df['mean_wap']=mean_wap
# train_df.to_csv(PATH/'train_with_wap.csv')

In [5]:
# np_data = data_df[['bid_price1', 'ask_price1', 'bid_size1', 'ask_size1','bid_price2', 'ask_price2', 'bid_size2', 'ask_size2']].to_numpy()

# del data_df

# np_data = np_data.astype('float32')
# torch_data = torch.tensor(np_data)

# del np_data

# torch.save(torch_data, PATH/'torch_data.pth')

In [6]:
%%time
torch_data = torch.load(PATH/'torch_data.pth')

for i in (2,3,6,7): torch_data[:,i] *= 1_000_000

CPU times: user 3.64 s, sys: 11.7 s, total: 15.3 s
Wall time: 1min 56s


In [7]:
bid_price1, ask_price1, bid_size1, ask_size1,bid_price2, ask_price2, bid_size2, ask_size2 = [torch_data[:,i] for i in range(8)]

In [8]:
wap = (bid_price1 * ask_size1 + ask_price1 * bid_size1)/(bid_size1 + ask_size1)
wap2 = (bid_price2 * ask_size2 + ask_price2 * bid_size2)/(bid_size2 + ask_size2)
wap_balance = wap - wap2
price_spread =  (ask_price1 - bid_price1) / ((ask_price1 + bid_price1)/2)
bid_spread = bid_price1 - bid_price2
ask_spread = ask_price1 = ask_price2
total_volume = ask_size1+ask_size2 +bid_size1 + bid_size2
volume_imbalance = (ask_size1 + ask_size2) - (bid_size1 + bid_size2)
feats= torch.stack([wap, wap2, wap_balance, price_spread, bid_spread, ask_spread, total_volume, volume_imbalance])
feats.shape

torch.Size([8, 257359200])

In [9]:
torch_data = torch.cat([torch_data, feats.T], dim=1)
torch_data.shape

torch.Size([257359200, 16])

In [10]:
del bid_price1, ask_price1, bid_size1, ask_size1,bid_price2, ask_price2, bid_size2, ask_size2,wap, wap2, wap_balance, price_spread, bid_spread, ask_spread, total_volume, volume_imbalance,feats

In [11]:
means, stds = torch_data.mean(dim=0), torch_data.std(dim=0)
print(means, stds)
torch_data = (torch_data - means) / stds

tensor([ 9.9968e-01,  1.0003e+00,  7.6999e+02,  7.6673e+02,  9.9948e-01,
         1.0005e+00,  9.5934e+02,  9.2822e+02,  1.0000e+00,  1.0000e+00,
         1.2723e-06,  6.6282e-04,  1.9751e-04,  1.0005e+00,  3.4243e+03,
        -3.4377e+01]) tensor([3.6881e-03, 3.6871e-03, 5.3541e+03, 4.9549e+03, 3.7009e-03, 3.6991e-03,
        6.6838e+03, 5.7353e+03, 3.6899e-03, 3.7075e-03, 5.1506e-04, 7.6403e-04,
        2.5716e-04, 3.6991e-03, 2.0439e+04, 6.1751e+03])


In [12]:
train_df = pd.read_csv(PATH/'train_with_wap.csv')

train_ftrs = pd.read_csv(PATH/'train_with_ftrs.csv')
train_ftrs.stock_id = train_ftrs.stock_id.astype('category')


train_ftrs = train_ftrs.drop(['stock_id_target_enc', 'row_id', 'target'], axis=1)
train_df = train_df.drop(['stock_id'], axis=1)

train_df = pd.concat([train_df, train_ftrs], axis=1)

In [13]:
w = train_df.mean_wap.to_numpy()

w =np.sort(w)

bins = []

step = (len(w)+9)//10
for i in range(0, len(w), step):
    j = min(i+step, len(w))
    bins.append(w[j] if j< len(w) else 2)
    print(w[i], w[j-1],np.median(w[i:j]) )

wap_bin = np.digitize(train_df.mean_wap, bins)

train_df['wap_bin']=wap_bin

0.9172531180048944 0.9971951138775056 0.9956492523836432
0.9971951327900604 0.9984703256953886 0.9979708959569917
0.9984703751656142 0.9991431543863876 0.9988451162091514
0.9991431683337992 0.999616836138266 0.9993924853218963
0.9996168497680276 1.0000141933324718 0.9998199779860442
1.0000141975255898 1.0004091980748968 1.00020793857736
1.0004091986325243 1.000876884050013 1.0006291062379966
1.0008769297217603 1.0015230362748495 1.0011664016312456
1.0015230933064834 1.002768629281105 1.0020189286622183
1.0027686365447124 1.077264872305151 1.004330146304397


In [14]:
w = train_df.target.to_numpy()

w =np.sort(w)

bins = []
bin_med=[]
step = (len(w)+9)//10
for i in range(0, len(w), step):
    j = min(i+step, len(w))
    bins.append(w[j] if j< len(w) else 1)
    bin_med.append(np.median(w[i:j]))
    print(w[i], w[j-1],np.median(w[i:j]) )

target_bin = np.digitize(train_df.target, bins)

train_df['target_bin']=target_bin

0.000105263 0.001436988 0.001175852
0.001436991 0.001838282 0.0016455530000000001
0.001838308 0.002210492 0.0020263215
0.002210493 0.002604845 0.00240206
0.002604847 0.003048064 0.002817908
0.003048071 0.003585089 0.0033027735
0.003585105 0.00428704 0.003905317
0.004287042 0.005311553 0.0047378975
0.005311559 0.007240946 0.0060829544999999995
0.007240976 0.07032062 0.009453786499999998


In [15]:
class ResBlock(nn.Module):
    def __init__(self, ch):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Conv1d(ch, ch, kernel_size = 5, padding = 2, padding_mode='replicate'),
            nn.BatchNorm1d(ch),
            nn.ReLU(),
            nn.Conv1d(ch, ch, kernel_size = 5, padding = 2, padding_mode='replicate'),
            nn.BatchNorm1d(ch),
        )
        
    def forward(self, x):
        res = self.layers(x) + x
        res = F.relu(res)
        return res

In [16]:
EMB_DIM = 32

class OptiverModel(nn.Module):
    def __init__(self, conv_in, n_feat, conv_chan=32):
        super().__init__()

        conv_out = 4 * conv_chan
        conv_layers = [nn.Conv1d(conv_in, conv_chan, kernel_size=1)]
        for _ in range(8):
            conv_layers += [ResBlock(conv_chan), ResBlock(conv_chan), ResBlock(conv_chan), ResBlock(conv_chan)
                       , nn.AvgPool1d(2, padding=1),
                      ]
        conv_layers += [Flatten()]#, nn.Dropout(.1)]   
        self.conv_layers = nn.Sequential(*conv_layers)
        
        self.stock_embedding = nn.Embedding(127,EMB_DIM )
        self.emb_drop = nn.Dropout(.25)
        class_inp =  EMB_DIM + n_feat  + conv_out
        self.classifier = nn.Sequential(
            LinBnDrop(class_inp, 400, p = 0, act = nn.ReLU()),
            LinBnDrop(400, 200, p=.25, act = nn.ReLU()), 
            LinBnDrop(200, 1, p=0),
            SigmoidRange(0, .1)
        )
        

    def forward(self, stock_id, features, book_data):
        book_feat = self.conv_layers(book_data)
        stock_emb = self.emb_drop(self.stock_embedding(stock_id)).squeeze()
        x = torch.cat([book_feat, stock_emb, features], dim=1)
        res = self.classifier(x)
        
        return res

In [17]:
class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.embeds = nn.Embedding(127, 23)
        self.bn_embs = nn.BatchNorm1d(23)
        self.layers = nn.Sequential(
            LinBnDrop(49,400,act=nn.ReLU(), lin_first=True),
            LinBnDrop(400,200, act=nn.ReLU(), lin_first=True),
            LinBnDrop(200,1,bn=False),
            SigmoidRange(0, .1)
        )
    def forward(self,  stock_id, feats):
        stock_emb = self.embeds(stock_id.squeeze())
        stock_emb = self.bn_embs(stock_emb)
        x = torch.cat([stock_emb, feats], dim=1)
        return self.layers(x)

In [18]:
def rmspe(preds, targs):
    x = (targs-preds)/targs
    return (x**2).mean().sqrt()

In [19]:
def rmspe_loss(preds, targs, reduction='mean'):
    assert reduction == 'mean'
    x = (targs-preds)/targs
    return (x**2).mean().sqrt()

In [20]:
class MyProc(TabularProc):
    def encodes(self, to):
        print(type(to))
        return to
    def __getitem__(self,k):
        print('getting', k)
        return 0

In [21]:
class ReadBatch(ItemTransform):
    def encodes(self, to):
        book_offsets = torch.tensor(to['offset'].to_numpy()).long()
        book_data = torch_data.view(-1,600,16)[book_offsets//600,:,:]
        book_data = book_data.permute(0,2,1)
        res = (tensor(to.cats).long(),tensor(to.conts).float(), book_data)        
        res = res + (tensor(to.targ),)
        if to.device is not None: res = to_device(res, to.device)
        return res

In [30]:
Normalize??

Init signature: Normalize(self, mean=None, std=None, axes=(0, 2, 3))
Source:        
class Normalize(DisplayedTransform):
    "Normalize/denorm batch of `TensorImage`"
    parameters,order = L('mean', 'std'),99
    def __init__(self, mean=None, std=None, axes=(0,2,3)): store_attr()

    @classmethod
    def from_stats(cls, mean, std, dim=1, ndim=4, cuda=True): return cls(*broadcast_vec(dim, ndim, mean, std, cuda=cuda))

    def setups(self, dl:DataLoader):
        if self.mean is None or self.std is None:
            x,*_ = dl.one_batch()
            self.mean,self.std = x.mean(self.axes, keepdim=True),x.std(self.axes, keepdim=True)+1e-7

    def encodes(self, x:TensorImage): return (x-self.mean) / self.std
    def decodes(self, x:TensorImage):
        f = to_cpu if x.device.type=='cpu' else noop
        return (x*f(self.std) + f(self.mean))

    _docs=dict(encodes="Normalize batch", decodes="Denormalize batch")
File:           /mnt/dysk25/repos/fastai/fastai/data/transforms.py
Type:  

In [32]:
@Normalize
def setups(self, to:Tabular):
    store_attr(but='to', means=dict(getattr(to, 'train', to).conts.mean()),
               stds=dict(getattr(to, 'train', to).conts.std(ddof=0)+1e-7))
    return self(to)

@Normalize
def encodes(self, to:Tabular):
    print('Normalizing')
    to.conts = (to.conts-self.means) / self.stds
    return to

@Normalize
def decodes(self, to:Tabular):
    to.conts = (to.conts*self.stds ) + self.means
    return to

In [33]:

cont_nn,cat_nn = cont_cat_split(train_ftrs, max_card=9000, dep_var='target')
procs_nn = [Categorify, FillMissing, Normalize]
to_nn = TabularPandas(train_df, procs_nn, cat_nn, cont_nn,
                      splits=ColSplitter()(train_df), y_names='target')
dls = to_nn.dataloaders(1024, after_batch = ReadBatch)

Normalizing


In [34]:
bx1, bx2,bx3, by = dls.one_batch()

bx1.shape, bx2.shape, bx3.shape, by.shape

(torch.Size([1024, 1]),
 torch.Size([1024, 26]),
 torch.Size([1024, 16, 600]),
 torch.Size([1024, 1]))

In [39]:
_=dls.valid.one_batch()

In [35]:
model = OptiverModel(16, 26).cuda()

In [36]:
model(bx1.cuda(),bx2.cuda(), bx3.cuda()).shape

torch.Size([1024, 1])

In [26]:
class ResnetModel(nn.Module):
    def __init__(self):
        self.conv_layers = None

In [27]:
def split_2way(model):
    #return L(params(model.initial_conv)+params(model.conv_layers), params(model.classifier))
    return L(params(model.conv_layers), params(model.classifier)+params(model.stock_embedding))

In [40]:
conv_layers = torch.load('resnet_feat.pth').conv_layers

model = OptiverModel(16, 26)
model.conv_layers = conv_layers

#learn = Learner(dls, ConvModel(),loss_func=mspe_loss, metrics=AccumMetric(rmspe))
learn = Learner(dls,model, loss_func=rmspe_loss, splitter = split_2way, metrics=AccumMetric(rmspe))

In [41]:
learn.fine_tune(100,1e-3, freeze_epochs=40)

epoch,train_loss,valid_loss,rmspe,time
0,7.997128,8.498618,8.541858,00:35
1,5.683980,5.037484,5.097029,00:35
2,4.316352,3.698126,3.788697,00:35
3,3.221061,2.500652,2.604899,00:35
4,2.302841,1.776741,1.868098,00:35
5,1.703205,1.348328,1.429101,00:35
6,1.069030,0.922054,0.961660,00:35
7,0.824780,0.880993,0.907036,00:35
8,0.635889,0.698549,0.728229,00:35
9,0.359029,0.362370,0.375438,00:34


epoch,train_loss,valid_loss,rmspe,time
0,0.239691,0.230648,0.233576,00:45
1,0.238779,0.229809,0.232953,00:45
2,0.238798,0.245833,0.275786,00:45
3,0.238319,0.230882,0.233795,00:45
4,0.242525,0.230494,0.233348,00:45
5,0.237314,0.233066,0.236073,00:45
6,0.236379,0.231086,0.234197,00:45
7,0.235916,0.230920,0.233745,00:45
8,0.236314,0.232918,0.236044,00:45
9,0.236832,0.231911,0.235006,00:45
